# Machine learning

## Imports

In [1]:
import sys
import cufflinks
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore')

sys.path.append('./..')
cufflinks.go_offline()

In [2]:
from Corpus.Corpus import get_corpus, filter_binary_pn, filter_corpus_small
from auxiliar.VectorizerHelper import vectorizer, vectorizerIdf, preprocessor
from auxiliar import parameters
from auxiliar.HtmlParser import HtmlParser

In [3]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import KFold
import copy

## Config

In [4]:
polarity_dim = 2
clasificadores=['lr', 'ls', 'mb', 'rf']
idf =  False
target_names=['Neg', 'Pos']
kfolds = 10
base_dir = '2-clases' if polarity_dim == 2 else ('3-clases' if polarity_dim == 3 else '5-clases')
name = 'machine_learning/tweeter/base_line'

## Get data

In [5]:
cine = HtmlParser(200, "http://www.muchocine.net/criticas_ultimas.php", 1)
data_corpus = get_corpus('general-corpus', 'general-corpus', 1, None)

if polarity_dim == 2:
    data_corpus = filter_binary_pn(data_corpus)
    cine = filter_binary_pn(cine.get_corpus())
elif polarity_dim == 3:
    data_corpus = filter_corpus_small(data_corpus)
    cine = filter_corpus_small(cine.get_corpus())
elif polarity_dim == 5:
    cine = cine.get_corpus()
cine = cine[:5000]
used_data = pd.DataFrame(data_corpus)
split = used_data.shape[0] * 0.7
data_corpus = None

#Intentando obtener datos del archivo csv...
./../Corpus/../data/general-corpus.csv
#Datos recuperados!


## Split data

In [6]:
used_data.reset_index().groupby('polarity').agg({'index': 'count'}).iplot(kind='bar')

In [7]:
train_corpus = used_data.loc[:split - 1 , :]
test_corpus = used_data.loc[split:, :]

## Initialize ML

In [8]:
vect = vectorizerIdf if idf else vectorizer
ls = CalibratedClassifierCV(LinearSVC()) if polarity_dim == 2 else OneVsRestClassifier(CalibratedClassifierCV(LinearSVC()))
lr = LogisticRegression(solver='lbfgs') if polarity_dim == 2 else OneVsRestClassifier(LogisticRegression())
mb = MultinomialNB() if polarity_dim == 2 else OneVsRestClassifier(MultinomialNB())
rf = RandomForestClassifier() if polarity_dim == 2 else OneVsRestClassifier(RandomForestClassifier())

In [9]:
pipeline_ls = Pipeline([
    ('vect', copy.deepcopy(vect)),
    ('ls', ls)
])
pipeline_lr = Pipeline([
    ('vect', copy.deepcopy(vect)),
    ('lr', lr)
])
pipeline_mb = Pipeline([
    ('vect', copy.deepcopy(vect)),
    ('mb', mb)
])
pipeline_rf = Pipeline([
    ('vect', copy.deepcopy(vect)),
    ('rf', rf)
])

In [10]:
pipelines = {
    'ls': pipeline_ls,
    'lr': pipeline_lr,
    'mb': pipeline_mb,
    'rf': pipeline_rf
}
pipelines_train = {
    'ls': ls,
    'lr': lr,
    'mb': mb,
    'rf': rf
}

## Train

In [11]:
folds = pd.read_pickle('../data/pkls/folds.pkl') # k-folds precargados
folds = folds.values

In [12]:
x_vect = vect.fit_transform(train_corpus.content, train_corpus.polarity).toarray()

In [13]:
cine_vect = vect.transform(cine.content).toarray()

In [14]:
vect.vocabulary_

{'marodriguezb': 10091,
 'Gracias': 2253,
 'MAR': 2893,
 'Off': 3407,
 'pensando': 11061,
 'regalito': 12017,
 'Sinde': 4222,
 'va': 13321,
 'SGAE': 4052,
 'cuando': 6864,
 'van': 13353,
 'sus': 12811,
 'corruptos': 6765,
 'Intento': 2518,
 'no': 10576,
 'sacar': 12332,
 'conclusiones': 6519,
 'intento': 9312,
 'Conozco': 1299,
 'adicto': 5068,
 'drama': 7542,
 'Ja': 2592,
 'ja': 9434,
 'suena': 12755,
 'algo': 5217,
 'Toca': 4447,
 'crackoviadeTV3': 6793,
 'Grabación': 2251,
 'dl': 7504,
 'especial': 8017,
 'Navideño': 3286,
 'Mari': 3009,
 'crismas': 6825,
 'Buen': 963,
 'día': 7589,
 'Lo': 2845,
 'primero': 11509,
 'mandar': 10011,
 'abrazo': 4916,
 'grande': 8728,
 'Miguel': 3119,
 'su': 12726,
 'familia': 8291,
 'libertadmontes': 9757,
 'Hoy': 2398,
 'podría': 11274,
 'ser': 12490,
 'grandeza': 8730,
 'humana': 8994,
 'Desde': 1534,
 'escaño': 7959,
 'Todo': 4453,
 'listo': 9791,
 'empezar': 7765,
 'endiascomohoy': 7826,
 'Congreso': 1293,
 'http': 8984,
 'co': 6347,
 'Mu2yIgCb': 

In [ ]:
results = {}
with tqdm(total=len(clasificadores) * 10) as pbar:
    for c in clasificadores:
        results[c] = { 'real': {}, 'cine_real': {}, 'predicted': {}, 'cine_predicted': {} }
        i = 0
        for train_index, test_index in folds:
            train_x = x_vect[train_index]
            train_y = train_corpus.polarity[train_index]
            test_x = x_vect[test_index]
            test_y = train_corpus.polarity[test_index]

            pipelines_train[c].fit(train_x, train_y)

            predicted = pipelines_train[c].predict(test_x)
            cine_pred = pipelines_train[c].predict(cine_vect)
            
            results[c]['real'][i] = test_y.values.tolist()
            results[c]['cine_real'][i] = cine.polarity.values.tolist()
            
            results[c]['predicted'][i] = predicted.tolist()
            results[c]['cine_predicted'][i] = cine_pred.tolist()
            i = i + 1

            pbar.update(1)

    

 85%|████████▌ | 34/40 [06:15<01:41, 16.93s/it]

In [ ]:
pd.DataFrame(results).to_pickle('../results/'+name+'/'+base_dir+'/results.pkl')